# Knapsack Solver Evaluation

**Objective:** Evaluate all solver results from `solver_runs.json` according to the protocol in `team_project_plan.md`.

**Protocol Steps:**
1.  Load all solver run data (`solver_runs.json`) into a pandas DataFrame.
2.  Determine the **Reference Value ($V_{known}$)** for all 27 instances.
    * For `n=50`, $V_{known}$ is the value from the exact solvers (DP, B&B).
    * For `n=200` & `n=500`, $V_{known}$ is the best value found across *all* algorithms and runs.
3.  Calculate the **Optimality Gap** for every single run: `gap = (V_known - V_run) / V_known`.
4.  Aggregate per-instance stats (for randomized algorithms) into `df_agg_instance`.
5.  Aggregate per-algorithm/config stats (median gap, IQR, etc.) into `df_summary`.
6.  Generate visualizations (boxplots for gap and runtime).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
from scipy.stats import iqr

# --- Configuration ---
DATA_ROOT = Path("knapsack_multisize_data")
RESULTS_DIR = Path("experiment_results")
PLOTS_DIR = RESULTS_DIR / "evaluation_plots"
PLOTS_DIR.mkdir(exist_ok=True)

RUNS_FILE = RESULTS_DIR / "solver_runs.json"

# --- IMPORTANT: Set your exact solver names here ---
# These names MUST match the 'method' names in solver_runs.json
# (as defined in the 'SOLVERS_TO_RUN' dict in the previous notebook)
# UPDATED: Added 'Backtracking' as per the project plan.
EXACT_SOLVERS = [
    "DynamicProgramming", 
    "BranchAndBound",
    "Backtracking"
]

print(f"Loading runs from: {RUNS_FILE}")
print(f"Saving plots to:   {PLOTS_DIR}")
print(f"Exact solvers defined as: {EXACT_SOLVERS}")

In [ ]:
# Load the JSON file with all solver runs
try:
    df_runs = pd.read_json(RUNS_FILE)
except ValueError as e:
    print(f"Error: Could not load {RUNS_FILE}. {e}")
    print("Please ensure 'solver_framework.ipynb' (or similar) was run successfully.")
    # Create empty DataFrame to avoid downstream errors
    df_runs = pd.DataFrame(columns=[
        'method', 'parameters', 'seed', 'run_index', 'instance_file',
        'instance_n', 'instance_dist', 'instance_cap_ratio_str',
        'instance_seed', 'best_value', 'runtime', 'logs'
    ])

if not df_runs.empty:
    print(f"Loaded {len(df_runs)} total runs across {df_runs['method'].nunique()} methods.")
    
    # --- Data Preprocessing ---
    
    # Check for and report any errors that occurred during the run
    # Errors were saved as 'best_value': None
    errors = df_runs['best_value'].isna().sum()
    if errors > 0:
        print(f"\nWARNING: Found {errors} runs that failed (had 'best_value' = None). These will be dropped.")
        print(df_runs[df_runs['best_value'].isna()][['method', 'instance_file', 'logs']])
        df_runs = df_runs[df_runs['best_value'].notna()] # Drop failed runs
    
    # Convert columns to categories for efficient filtering/grouping
    df_runs['instance_n'] = df_runs['instance_n'].astype('category')
    df_runs['instance_dist'] = df_runs['instance_dist'].astype('category')
    df_runs['method'] = df_runs['method'].astype('category')

    print("\nDataFrame Info:")
    df_runs.info()
    
    print("\nDataFrame Head:")
    display(df_runs.head())
else:
    print("\nDataFrame is empty. Cannot proceed with evaluation.")

## 2. Determine Reference Values ($V_{known}$)

Here we establish the ground-truth optimal or best-known value for each of the 27 instances based on the evaluation protocol.

In [ ]:
v_known_map = pd.Series(dtype=float)

if not df_runs.empty:
    # --- Step 1: Get V_known for n=50 from exact solvers ---
    df_n50 = df_runs[df_runs['instance_n'] == 50]
    df_n50_exact = df_n50[df_n50['method'].isin(EXACT_SOLVERS)]
    
    if df_n50_exact.empty:
        print("WARNING: No data found for n=50 using 'EXACT_SOLVERS'. Check your solver names.")
        print("Using best-known value from *all* n=50 solvers as a fallback.")
        v_known_n50 = df_n50.groupby('instance_file')['best_value'].max()
    else:
        v_known_n50 = df_n50_exact.groupby('instance_file')['best_value'].max()
        print(f"Found {len(v_known_n50)} reference values for n=50 from exact solvers.")
        
        # --- Optional Verification: Check if exact solvers agree ---
        if len(EXACT_SOLVERS) > 1:
            check = df_n50_exact.groupby('instance_file')['best_value'].nunique()
            if (check > 1).any():
                print("\n!! VERIFICATION WARNING: Your exact solvers DISAGREE on n=50 !!")
                disagree_files = check[check > 1].index
                print(df_n50_exact[df_n50_exact['instance_file'].isin(disagree_files)][['method', 'instance_file', 'best_value']])
            else:
                print("Verification: All exact solvers agree on n=50 values. Good.")

    # --- Step 2: Get V_known for n=200 & n=500 from *all* runs ---
    df_n_large = df_runs[df_runs['instance_n'].isin([200, 500])]
    v_known_n_large = df_n_large.groupby('instance_file')['best_value'].max()
    print(f"Found {len(v_known_n_large)} best-known values for n=200 & n=500 from all runs.")

    # --- Step 3: Combine into one map and merge into main DataFrame ---
    v_known_map = pd.concat([v_known_n50, v_known_n_large])
    v_known_map.name = "v_known"
    
    df_runs = df_runs.merge(v_known_map, left_on='instance_file', right_index=True)
    
    print(f"\nTotal instances with reference values: {df_runs['instance_file'].nunique()}")
    display(v_known_map.to_frame().head())
else:
    print("DataFrame empty, skipping $V_{known}$ calculation.")

## 3. Calculate Optimality Gap (Per-Run)

Now we compute the relative gap for every single solver run.

In [ ]:
if 'v_known' in df_runs.columns:
    # Calculate gap: (V_known - V_run) / V_known
    numerator = df_runs['v_known'] - df_runs['best_value']
    denominator = df_runs['v_known']
    
    # Handle V_known == 0 case to avoid division by zero
    df_runs['gap'] = np.where(
        denominator == 0,
        0.0, # If V_known is 0, gap is 0
        numerator / denominator
    )
    
    # Handle any other weirdness (e.g., V_run > V_known, which would be a negative gap)
    df_runs['gap'] = df_runs['gap'].fillna(0.0)
    
    print("Gap calculation complete. Descriptive stats for 'gap':")
    display(df_runs['gap'].describe())
    
    # Check for negative gaps (solver found a *better* value than V_known)
    # This should only happen if an exact solver failed or wasn't run on n=50
    if (df_runs['gap'] < -1e-9).any(): # Use tolerance for float precision
        print("\n!! WARNING: Found runs with a negative gap (better than V_known) !!")
        display(df_runs[df_runs['gap'] < -1e-9][[
            'method', 'instance_file', 'instance_n', 'best_value', 'v_known', 'gap'
        ]])
else:
    print("Column 'v_known' not found. Skipping gap calculation.")

## 4. Aggregate Per-Instance Statistics

Here we collapse the 10 runs for randomized algorithms into summary statistics (mean, median, best, std) *for each instance*. Deterministic solvers will just have 1 run, so their std will be 0.

In [ ]:
df_agg_instance = pd.DataFrame()

if 'gap' in df_runs.columns:
    # Define the columns to group by for a unique (solver, instance) pair
    group_cols = [
        'method', 'parameters', 'instance_file', 
        'instance_n', 'instance_dist', 'instance_cap_ratio_str'
    ]

    # Define the aggregations
    aggregations = {
        # Gap stats
        'gap': ['mean', 'median', 'std', 
                ('best', 'min'), # 'best' gap is the minimum gap
                ('worst', 'max')], 
        
        # Value stats
        'best_value': ['mean', 'median', 'std', 
                       ('best', 'max'), # 'best' value is the maximum value
                       ('worst', 'min')],
        
        # Runtime stats
        'runtime': ['mean', 'median', 'std', 'sum'],
        
        # Count of runs
        'run_index': ['count']
    }

    df_agg_instance = df_runs.groupby(group_cols).agg(aggregations)

    # Flatten the MultiIndex columns (e.g., ('gap', 'mean') -> 'gap_mean')
    df_agg_instance.columns = ['_'.join(col).strip() for col in df_agg_instance.columns.values]
    
    # Rename 'gap_best' to 'gap_best_run' to avoid confusion with 'best_value_best_run'
    df_agg_instance = df_agg_instance.rename(columns={
        'gap_best': 'gap_best_run',
        'gap_worst': 'gap_worst_run',
        'best_value_best': 'best_value_best_run',
        'best_value_worst': 'best_value_worst_run',
        'run_index_count': 'n_runs'
    })

    # Fill std=NaN with 0 (for deterministic solvers with 1 run)
    std_cols = [col for col in df_agg_instance.columns if col.endswith('_std')]
    df_agg_instance[std_cols] = df_agg_instance[std_cols].fillna(0.0)

    df_agg_instance = df_agg_instance.reset_index()

    print(f"Created per-instance aggregate table with {len(df_agg_instance)} rows.")
    display(df_agg_instance.head())
else:
    print("Column 'gap' not found. Skipping instance aggregation.")

## 5. Final Summary Table (Per-Algorithm)

This is the final report. We aggregate the per-instance statistics to get a high-level summary for each algorithm across each (n, distribution) pair.

We report **median $\pm$ IQR** and **mean $\pm$ std** for the **median gap** (i.e., the `gap_median` from the table above).

In [ ]:
df_summary = pd.DataFrame()

if not df_agg_instance.empty:
    # Group by method, size (n), and distribution
    summary_group_cols = ['method', 'instance_n', 'instance_dist']

    # Define aggregations for the final summary table
    summary_aggregations = {
        # Stats on the 'median_gap' from the 10 runs
        'gap_median': ['mean', 'std', 'median', iqr],
        
        # Stats on the 'best_gap' from the 10 runs
        'gap_best_run': ['mean', 'median'],
        
        # Stats on the 'mean_runtime' from the 10 runs
        'runtime_mean': ['mean', 'std', 'median', 'sum']
    }

    df_summary = df_agg_instance.groupby(summary_group_cols).agg(summary_aggregations)

    # Flatten column names (e.g., 'gap_median_mean')
    df_summary.columns = ['_'.join(col).strip() for col in df_summary.columns.values]
    
    # Rename for clarity
    df_summary = df_summary.rename(columns={
        'gap_median_mean': 'gap_median_of_medians',
        'gap_median_std': 'gap_std_of_medians',
        'gap_median_median': 'gap_median_of_medians_final',
        'gap_median_iqr': 'gap_iqr_of_medians'
    })
    
    df_summary = df_summary.reset_index()

    print("--- Final Evaluation Summary ---\n")
    
    # Display with nice formatting
    float_format = "{:.5f}".format
    display(df_summary.style.format({
        'gap_median_of_medians': float_format,
        'gap_std_of_medians': float_format,
        'gap_median_of_medians_final': float_format,
        'gap_iqr_of_medians': float_format,
        'gap_best_run_mean': float_format,
        'gap_best_run_median': float_format,
        'runtime_mean_mean': "{:.3f}".format,
        'runtime_mean_std': "{:.3f}".format,
        'runtime_mean_median': "{:.3f}".format,
        'runtime_mean_sum': "{:.1f}".format
    }))
    
    # --- Save Summary to CSV ---
    summary_path = RESULTS_DIR / "evaluation_summary.csv"
    try:
        df_summary.to_csv(summary_path, index=False, float_format="%.6f")
        print(f"\nSummary table saved to {summary_path}")
    except Exception as e:
        print(f"\nError saving summary CSV: {e}")
else:
    print("Aggregated instance DataFrame is empty. Skipping final summary.")

## 6. Visualization: Optimality Gap

Visualize the distribution of optimality gaps for each solver. We will plot the **best gap** (`gap_best_run`) achieved for each instance.

In [ ]:
if not df_agg_instance.empty:
    print("Generating Optimality Gap boxplots...")
    
    # Use seaborn's catplot (kind='box') for a faceted view
    g = sns.catplot(
        data=df_agg_instance,
        x='method',
        y='gap_best_run', # Plotting the 'best run' gap per instance
        col='instance_dist',  # Separate columns by distribution
        row='instance_n',     # Separate rows by size (n)
        kind='box',
        height=4, 
        aspect=1.5,
        margin_titles=True,
        sharey=False # Gaps might be on very different scales
    )

    # Use a symmetric log scale for 'gap' because it can be 0 or very close to 0
    g.set(yscale="symlog", linthresh=1e-5)
    g.set_xticklabels(rotation=45, ha='right')
    g.set_axis_labels("Solver Method", "Best Optimality Gap (symlog scale)")
    g.fig.suptitle("Optimality Gap (Best Run) by Solver, Size, and Distribution", y=1.03)

    # Save the plot
    plot_path = PLOTS_DIR / "gap_boxplot_faceted.png"
    g.savefig(plot_path, dpi=150, bbox_inches="tight")
    print(f"Gap plot saved to {plot_path}")
    plt.show()
else:
    print("DataFrame empty. Skipping gap visualization.")

## 7. Visualization: Runtime

Visualize the distribution of execution time for each solver. We will plot the **mean runtime** (`runtime_mean`) on a **log scale**.

In [ ]:
if not df_agg_instance.empty:
    print("Generating Runtime boxplots...")
    
    g = sns.catplot(
        data=df_agg_instance,
        x='method',
        y='runtime_mean', # Plotting the 'mean run' time per instance
        col='instance_dist', 
        row='instance_n', 
        kind='box',
        height=4, 
        aspect=1.5,
        margin_titles=True,
        sharey=False # Runtimes will vary greatly by n
    )

    # Use a log scale for runtime, as it spans orders of magnitude
    g.set(yscale="log") 
    g.set_xticklabels(rotation=45, ha='right')
    g.set_axis_labels("Solver Method", "Mean Runtime (log scale, seconds)")
    g.fig.suptitle("Mean Solver Runtime by Size and Distribution", y=1.03)

    # Save the plot
    plot_path = PLOTS_DIR / "runtime_boxplot_faceted.png"
    g.savefig(plot_path, dpi=150, bbox_inches="tight")
    print(f"Runtime plot saved to {plot_path}")
    plt.show()
else:
    print("DataFrame empty. Skipping runtime visualization.")

## 8. (Placeholder) Visualization: Convergence Plots

This section is a **placeholder** to show how you *would* plot convergence, as requested in the plan.

It assumes that your randomized solvers (GA, SA, ACO, etc.) stored an array or list called `convergence_history` inside the `logs` dictionary for each run.

In [ ]:
# --- This is a template, you must customize it ---

# 1. Define the solver and instance you want to inspect
SOLVER_TO_PLOT = "GeneticAlgorithm" # <-- CHANGE THIS to your solver name
INSTANCE_FILE_TO_PLOT = "knapsack_n500_seed20251025.json" # <-- CHANGE THIS to an instance file

if 'gap' not in df_runs.columns:
    print("df_runs not available. Skipping convergence plot.")
else:
    # 2. Filter df_runs to get the 10 runs for this (solver, instance) pair
    df_plot = df_runs[
        (df_runs['method'] == SOLVER_TO_PLOT) &
        (df_runs['instance_file'] == INSTANCE_FILE_TO_PLOT)
    ]

    if df_plot.empty:
        print(f"No data found for {SOLVER_TO_PLOT} on {INSTANCE_FILE_TO_PLOT}")
        print("Check your solver names and instance file names.")
    else:
        print(f"Found {len(df_plot)} runs to plot for {SOLVER_TO_PLOT}...")
        
        plt.figure(figsize=(12, 7))
        
        v_known_for_instance = df_plot['v_known'].iloc[0]
        
        # 3. Loop over each run and plot its history
        for idx, run in df_plot.iterrows():
            # 4. *** This is the critical part ***
            # You must know the key inside your 'logs' dict.
            # We assume it's 'convergence_history' for this example.
            history = run['logs'].get('convergence_history') 
            
            if history and isinstance(history, list):
                # Plot value
                plt.plot(history, label=f"Run {run['run_index']}", alpha=0.6)
                # To plot gap instead:
                # gap_history = [(v_known_for_instance - v) / v_known_for_instance for v in history]
                # plt.plot(gap_history, label=f"Run {run['run_index']}", alpha=0.6)
            else:
                print(f"  -> Run {run['run_index']}: 'convergence_history' key not found or not a list in logs.")
        
        # Plot the known best value as a dashed line
        plt.axhline(y=v_known_for_instance, color='red', linestyle='--', label=f"V_known = {v_known_for_instance}")
        
        plt.title(f"Convergence Plot: {SOLVER_TO_PLOT}\nInstance: {INSTANCE_FILE_TO_PLOT}")
        plt.xlabel("Iteration / Generation")
        plt.ylabel("Best Value Found")
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        
        # Save plot
        plot_path = PLOTS_DIR / f"convergence_{SOLVER_TO_PLOT}_n{df_plot['instance_n'].iloc[0]}.png"
        plt.savefig(plot_path, dpi=150, bbox_inches="tight")
        print(f"Convergence plot saved to {plot_path}")
        
        plt.show()
